In [3]:
from google.colab import files
files.upload()

Saving archive.zip to archive.zip


In [30]:
import os
import zipfile
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf


zip_path = 'archive.zip'
extract_folder = 'FER2013'



with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

img_width, img_height = 48, 48
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


def load_all_images(data_dir):
    images = []
    labels = []
    for emotion_index, emotion in enumerate(emotions):
        emotion_dir = os.path.join(data_dir, emotion)
        if os.path.exists(emotion_dir):
            for filename in os.listdir(emotion_dir):
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    img_path = os.path.join(emotion_dir, filename)
                    img = load_img(img_path, target_size=(img_width, img_height), color_mode='grayscale')
                    img_array = img_to_array(img)
                    images.append(img_array)
                    labels.append(emotion_index)
    return np.array(images), np.array(labels)



X_data, y_data = load_all_images(os.path.join(extract_folder, 'train'))
X_data_test, y_data_test = load_all_images(os.path.join(extract_folder, 'test'))


X = np.concatenate((X_data, X_data_test), axis=0)
y = np.concatenate((y_data, y_data_test), axis=0)


X = X.astype('float32') / 255


y = tf.keras.utils.to_categorical(y, num_classes=len(emotions))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(emotions), activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


model.save('facial_expression_model.h5')
print("Model saved successfully")


Epoch 1/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 99s 109ms/step - accuracy: 0.2744 - loss: 1.7769 - val_accuracy: 0.4189 - val_loss: 1.5472
Epoch 2/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 142s 109ms/step - accuracy: 0.4038 - loss: 1.5396 - val_accuracy: 0.4667 - val_loss: 1.4117
Epoch 3/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 140s 107ms/step - accuracy: 0.4527 - loss: 1.4235 - val_accuracy: 0.4815 - val_loss: 1.3656
Epoch 4/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 141s 106ms/step - accuracy: 0.4775 - loss: 1.3675 - val_accuracy: 0.4929 - val_loss: 1.3235
Epoch 5/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 145s 109ms/step - accuracy: 0.4966 - loss: 1.3020 - val_accuracy: 0.5028 - val_loss: 1.3081
Epoch 6/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 95s 105ms/step - accuracy: 0.5130 - loss: 1.2704 - val_accuracy: 0.5104 - val_loss: 1.2909
Epoch 7/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 99s 110ms/step - accuracy: 0.5307 - loss: 1.2205 - val_accuracy: 0.5199 - val_loss: 1.2846
Epoch 8/10
898/898 ━━━━━━━━━━━━━━━━━━━━ 140s 108ms/step - accuracy: 0.5539 - lo

Model saved successfully


In [35]:
!pip install ipywidgets

import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from google.colab import files



uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False
)


display(uploader)


def handle_upload(change):
    uploaded_file = next(iter(uploader.value.values()))
    content = uploaded_file['content']


    with open('uploaded_image.jpg', 'wb') as f:
        f.write(content)


    image_path = 'uploaded_image.jpg'
    predicted_emotion, probabilities = predict_emotion(image_path)

    print("Predicted emotion:", predicted_emotion)
    print("Probabilities:", probabilities)


    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.title(f"Predicted Emotion: {predicted_emotion}")
    plt.axis('off')
    plt.show()


uploader.observe(handle_upload, names='value')

FileUpload(value={}, accept='image/*', description='Upload')